## Import libraries

In [2]:
# add the path to my spark(you may not need these lines): 
import sys
sys.path.append('/home/victor/Downloads/spark-2.3.1-bin-hadoop2.7/python')
sys.path.append('/home/victor/Desktop/TILES/src/pyspark_scripts')


In [3]:
from loadCSV import *
from MLModels import MLModel
from PySparkDFGetWindows import getWindows
from computeStats import computeStats
from datetime import timedelta, datetime
import pyspark
import os
from shutil import copyfile
%load_ext autoreload
%autoreload 2

## Path to the folder which contains all the subfolders(1,2,3,4)

In [4]:
# !!!!Modify the path!!!!!!
path = '/home/victor/Desktop/Data/keck_wave2_gdrive'
outputPath = '/home/victor/Desktop/results'
if not os.path.isdir(outputPath):
    os.mkdir(outputPath)
print (os.listdir(path) )
print ('\nyour output should looks like this: 1_raw_data, 2_raw_csv, 3_preprocessed_data, 4_extracted_features')

['AWS Downloading the data.docx', 'TheBigDF', '4_extracted_features', '3_preprocessed_data', 'intermediateData', '1_raw_data', '2_raw_csv']

your output should looks like this: 1_raw_data, 2_raw_csv, 3_preprocessed_data, 4_extracted_features


## Initialize Spark

In [5]:
# Init Spark
sqlContext = initSpark()

## Start horizontally merging data one by one

       Merge RealizeD 

In [49]:
# RealizeD is in the fourth folder
df = loadAndAggregateDataFromDir(path+'/'+'4_extracted_features', ["realizd"])
df.printSchema()
df.repartition(1).write.mode('overwrite').csv(path+'/intermediateData'+'/realizeD', header="true")
del df
print ('Saving....Done!!')

Data loading completed at:  1533161249.5963514
Verical Merge completed at:  1533161249.6656828
Data load and aggregation complete!!!
root
 |-- Timestamp: timestamp (nullable = true)
 |-- SecondsOnPhone: double (nullable = true)
 |-- ID: string (nullable = false)
 |-- date: date (nullable = true)

Saving....Done!!


    Merge phone events

In [50]:
# phone_event is in the fourth folder
df = loadAndAggregateDataFromDir(path+'/'+'4_extracted_features', ["phone_events"])
df.printSchema()
df.repartition(1).write.mode('overwrite').csv(path+'/intermediateData'+'/phone_events', header="true")
del df
print ('Saving....Done!!')

Data loading completed at:  1533161315.0065656
Verical Merge completed at:  1533161327.7242188
Data load and aggregation complete!!!
root
 |-- Timestamp: timestamp (nullable = true)
 |-- ssid: string (nullable = true)
 |-- wifi_mac: string (nullable = true)
 |-- phone_battery: integer (nullable = true)
 |-- gps_lat: string (nullable = true)
 |-- gps_lon: string (nullable = true)
 |-- ID: string (nullable = false)
 |-- date: date (nullable = true)

Saving....Done!!


    Merge app_surveys

In [51]:
# app surveys is in the fourth folder
df = loadAndAggregateDataFromDir(path+'/'+'4_extracted_features', ["tiles_app_survey"])
df.printSchema()
df.repartition(1).write.mode('overwrite').csv(path+'/intermediateData'+'/tiles_app_survey', header="true")
del df
print ('Saving....Done!!')

Data loading completed at:  1533161351.3306835
Verical Merge completed at:  1533161369.2070286
Data load and aggregation complete!!!
root
 |-- Timestamp: string (nullable = true)
 |-- surveyType: string (nullable = true)
 |-- durationInSeconds: string (nullable = true)
 |-- workBeforeSurvey: string (nullable = true)
 |-- onPhoneBeforeSurvey: string (nullable = true)
 |-- interactWithPeopleBeforeSurvey: string (nullable = true)
 |-- sportBeforeSurvey: string (nullable = true)
 |-- shoppingBeforeSurvey: string (nullable = true)
 |-- diningBeforeSurvey: string (nullable = true)
 |-- householingBeforeSurvey: string (nullable = true)
 |-- familyBeforeSurvey: string (nullable = true)
 |-- personalActivityBeforeSurvey: string (nullable = true)
 |-- educationBeforeSurvey: string (nullable = true)
 |-- transportBeforeSurvey: string (nullable = true)
 |-- orgActivityBeforeSurvey: string (nullable = true)
 |-- unKnownBeforeSurvey: string (nullable = true)
 |-- psycFlexPositiveEmotionasBeforeSurve

    Merge app_analytics

In [52]:
# app surveys is in the fourth folder
df = loadAndAggregateDataFromDir(path+'/'+'4_extracted_features', ["tiles_app_analytics"])
df.printSchema()
df.repartition(1).write.mode('overwrite').csv(path+'/intermediateData'+'/tiles_app_analytics', header="true")
del df
print ('Saving....Done!!')

Data loading completed at:  1533161384.2089138
Verical Merge completed at:  1533161384.2090068
Data load and aggregation complete!!!
root
 |-- uid: string (nullable = true)
 |-- Activity_Feed_opened: integer (nullable = true)
 |-- App_started: integer (nullable = true)
 |-- Contact_opened: integer (nullable = true)
 |-- FAQ_opened: integer (nullable = true)
 |-- Facebook_Auth: integer (nullable = true)
 |-- Fitbit_Auth: integer (nullable = true)
 |-- Instagram_Auth: integer (nullable = true)
 |-- Internal_survey_opened: integer (nullable = true)
 |-- Push_instructions_opened: integer (nullable = true)
 |-- QR_code_opened: integer (nullable = true)
 |-- Regular_push_opened: integer (nullable = true)

Saving....Done!!


    Merge fitbit 

In [53]:
#dfFit = sqlContext.read.csv(dataPathPre + '/fitbit/*Summary.csv', header = "true").repartition(6)
fitbitPath = path + '/3_preprocessed_data/fitbit/'
dfFitList = [] # a dict to hold all the temp dfs; each creted from one file
i = 0
listDir = sorted(os.listdir(fitbitPath))    
for filename in listDir:
    # If file is not csv: skip, Else create a df
    if not(filename.endswith(".csv")) or "Summary" not in filename:
        continue
    dfTemp = sqlContext.read.csv(fitbitPath + filename, header = "true")

    # If ID is there in the filename, strip it out and add a column to DF for it
    if '-' in filename:
        x = filename.find('-')
        ID = filename[x-8:x+28]
        #print(ID)
        dfTemp = dfTemp.withColumn("ID", lit(ID))

    # If there is Timestamp in df then convert it to date
    if "Timestamp" in dfTemp.columns:
        dfTemp = dfTemp.withColumn('date', to_date(dfTemp.Timestamp))
    dfFitList.append(dfTemp)
    i = i+1
    
# Vertical merge all the DF in the list    
dfFit = reduce(lambda a,b: a.union(b), dfFitList)
dfFit.repartition(1).write.mode('overwrite').csv(path+'/intermediateData'+'/fitbit', header="true")
del dfFit
print ('Saving....Done!!')

Saving....Done!!


    Merge omSignal

In [54]:
df = loadAndAggregateDataFromDir(path+'/'+'3_preprocessed_data', ["omsignal"])
df.printSchema()
df.repartition(1).write.mode('overwrite').csv(path+'/intermediateData'+'/omsignal', header="true")
del df
print ('Saving....Done!!')

Data loading completed at:  1533162477.070927
Verical Merge completed at:  1533162500.391785
Data load and aggregation complete!!!
root
 |-- Timestamp: timestamp (nullable = true)
 |-- BreathingDepth: double (nullable = true)
 |-- BreathingRate: double (nullable = true)
 |-- Cadence: double (nullable = true)
 |-- HeartRate: double (nullable = true)
 |-- Intensity: double (nullable = true)
 |-- Steps: double (nullable = true)
 |-- AngleFromVertical_rad: string (nullable = true)
 |-- AvgBreathingDepth: string (nullable = true)
 |-- AvgBreathingRate: string (nullable = true)
 |-- AvgGForce: string (nullable = true)
 |-- AvgHeartRate: string (nullable = true)
 |-- AvgXAccel_g: string (nullable = true)
 |-- AvgYAccel_g: string (nullable = true)
 |-- AvgZAccel_g: string (nullable = true)
 |-- Sitting: string (nullable = true)
 |-- Supine: string (nullable = true)
 |-- LowGCoverage: string (nullable = true)
 |-- RMSStdDev_ms: string (nullable = true)
 |-- RRPeakCoverage: string (nullable = tr

    Merge Owl_in_one

In [55]:
df = loadAndAggregateDataFromDir(path+'/'+'4_extracted_features', ["owl_in_one"])
df.printSchema()
df.repartition(1).write.mode('overwrite').csv(path+'/intermediateData'+'/owl_in_one', header="true")
del df
print ('Saving....Done!!')

Data loading completed at:  1533164160.3325465
Verical Merge completed at:  1533164162.0514112
Data load and aggregation complete!!!
root
 |-- Timestamp: timestamp (nullable = true)
 |-- NearestProximity: string (nullable = true)
 |-- ProximityCertainty: double (nullable = true)
 |-- AtNursingStation: double (nullable = true)
 |-- AtPatientRoom: double (nullable = true)
 |-- AtLounge: double (nullable = true)
 |-- AtMedicineRoom: double (nullable = true)
 |-- AtLabRoom: double (nullable = true)
 |-- AtReceivingRoom: double (nullable = true)
 |-- ID: string (nullable = false)
 |-- date: date (nullable = true)

Saving....Done!!


    Merge minew

In [ ]:
df = loadAndAggregateDataFromDir(path+'/'+'4_extracted_features', ["minew"])
df.printSchema()
df.repartition(1).write.mode('overwrite').csv(path+'/intermediateData'+'/minew', header="true")
del df
print ('Saving....Done!!')

    Merge Ground Truth Data

In [16]:
dataPath = path + '/' + '3_preprocessed_data' + '/' +'Ground_Truth'
idMap = CsvToDF( dataPath + '/'+ 'IDs.csv')
igtb = CsvToDF(dataPath +'/' + 'IGTB.csv')
mgt = CsvToDF(dataPath + '/' + 'MGT.csv')
# merge igtb and mgt
merged = igtb.df.join (mgt.df,['timestamp', 'uid'], how = 'full')
# replace uid by participant id 
merged = merged.join(idMap.df, idMap.df.user_id == merged.uid , how = 'inner')
merged = merged.drop("uid", 'user_id')
merged.repartition(1).write.mode('overwrite').csv(path+'/intermediateData'+'/Ground_Truth', header="true")
del merged
print ('Saving....Done!!')

Saving....Done!!


## All data is ready for outter join 

In [7]:
intermediate = path+'/intermediateData'
folderNames = os.listdir(intermediate)
print (folderNames)

['tiles_app_survey', 'tiles_app_analytics', 'phone_events', 'omsignal', 'minew', 'realizeD', 'fitbit', 'Ground_Truth', 'owl_in_one']


In [10]:
stack = []
for folder in folderNames:
    #get csv file name
    fileName = [file  for file in os.listdir(intermediate + '/' + folder) if file.endswith('.csv')][0]
    if folder in ['fitbit']:
        #copy and paste
        copyfile(intermediate+ '/' + folder + '/' + fileName, outputPath+'/'+fileName)
    elif folder in ['tiles_app_analytics']:
        temp1 = CsvToDF(intermediate+ '/' + folder + '/' + fileName)
    elif folder in ['Ground_Truth']:
        temp2 = CsvToDF(intermediate+ '/' + folder + '/' + fileName)
    else:
        #load in data one by one
        stack.append(CsvToDF(intermediate+ '/' + folder + '/' + fileName))

## Get the big time-series data frame

In [39]:
# start merging data 
index = 0
myDf = stack[0]
while index < len(stack):
    myDf = fullOuterJoinOnTimestamp(myDf, stack[index])
    index += 1

In [40]:
# dumpt to disk
stack[0].repartition(1).write.mode('overwrite').csv(outputPath + '/' + 'others', header="true")
print ('Done!')

Done!


## Get the ID data frame

In [11]:
idFile = temp1.df.join (temp2.df,temp1.df.uid == temp2.df.OMuser_id, how = 'full')

In [12]:
idFile.repartition(1).write.mode('overwrite').csv(outputPath + '/' + 'idFile', header="true")
print ('Done!')

Done!
